In [3]:
import pandas as pd
import numpy as np
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Dropout, Layer, MultiHeadAttention, LayerNormalization, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc # Import garbage collector
import sys
from io import StringIO
import pickle # Import the pickle library for saving objects


In [5]:
import pandas as pd
import numpy as np
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Dropout, Layer, MultiHeadAttention, LayerNormalization, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc # Import garbage collector
import sys
from io import StringIO
import pickle # Import the pickle library for saving objects

# Define the single buffer size to use
BUFFER_METERS = 500

# ==================== 1. Load Data ==================== #
orig = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")

drop_cols = ['Stations','River','Lat','Long','geometry']
numeric_cols = orig.drop(columns=drop_cols).columns.drop('RI')

# Train-test split
train_orig = orig.sample(10, random_state=42)
test_orig = orig.drop(train_orig.index)
train_combined = pd.concat([river_100, train_orig], ignore_index=True)

# ==================== 2. Collect ALL Rasters ==================== #
# We are not using rasters in this GNN-MLP model, but the paths are still
# defined for consistency with previous versions.
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")

print("Note: Raster data is not used in this GNN-MLP model.")

# ==================== 3. Create a Custom Data Generator ==================== #
class DataGenerator(Sequence):
    def __init__(self, mlp_data, gnn_data, y, batch_size=4, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        # Get batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Get batch data
        batch_mlp = self.mlp_data[batch_indices]
        batch_gnn = self.gnn_data[batch_indices, :]
        batch_y = self.y[batch_indices]
        
        return (batch_mlp, batch_gnn), batch_y

# ==================== 4. Prepare GNN & MLP Input (only once) ==================== #
coords_train = train_combined[['Long','Lat']].values
coords_test = test_orig[['Long','Lat']].values
dist_mat_train = distance_matrix(coords_train, coords_train)
gnn_train = np.exp(-dist_mat_train/10)
dist_mat_test_train = distance_matrix(coords_test, coords_train)
gnn_test = np.exp(-dist_mat_test_train/10)

scaler = StandardScaler()
mlp_train = scaler.fit_transform(train_combined[numeric_cols])
mlp_test = scaler.transform(test_orig[numeric_cols])
y_train = train_combined['RI'].values
y_test = test_orig['RI'].values

# ==================== 5. Define GNN-MLP Fusion Autoencoder Model ==================== #
def build_gnn_mlp_autoencoder_model(mlp_dim, gnn_dim):
    # Inputs for all branches
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    
    # --- Encoder Branch (MLP) ---
    mlp_encoded = Dense(128, activation="relu")(mlp_input)
    mlp_encoded = Dense(64, activation="relu", name="mlp_encoder_output")(mlp_encoded)

    # --- Encoder Branch (GNN) ---
    gnn_encoded = Dense(128, activation="relu")(gnn_input)
    gnn_encoded = Dense(64, activation="relu", name="gnn_encoder_output")(gnn_encoded)

    # --- Bottleneck/Latent Space ---
    # Concatenate the encoded representations
    latent_space = Concatenate(name="latent_space")([mlp_encoded, gnn_encoded])
    
    # --- Decoder Branch for Prediction ---
    # The decoder takes the latent space and performs the final prediction
    f = Dense(128, activation="relu")(latent_space)
    f = Dropout(0.4)(f)
    f = Dense(64, activation="relu")(f)
    output = Dense(1, activation="linear", name="final_output")(f)

    # Build and compile the model
    model = Model(inputs=[mlp_input, gnn_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

def evaluate_model(model, mlp_test, gnn_test_matrix, y_test, return_preds=False):
    """
    Evaluates the model on given data and returns R², RMSE, and predictions.
    """
    y_pred = model.predict((mlp_test, gnn_test_matrix)).flatten()
    
    if return_preds:
        return y_pred
    else:
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        return r2, rmse

def calculate_permutation_importance(model, mlp_data, gnn_data, y_true):
    """
    Calculates permutation feature importance for the MLP and GNN branches.
    """
    print("\nStarting Permutation Feature Importance Analysis...")
    # Get baseline R² on the unshuffled data
    baseline_r2, _ = evaluate_model(model, mlp_data, gnn_data, y_true)
    print(f"Baseline R² on test set: {baseline_r2:.4f}")

    importance = {}
    
    # Permute MLP input
    shuffled_mlp_data = mlp_data.copy()
    np.random.shuffle(shuffled_mlp_data)
    shuffled_r2, _ = evaluate_model(model, shuffled_mlp_data, gnn_data, y_true)
    importance['MLP'] = baseline_r2 - shuffled_r2

    # Permute GNN input
    shuffled_gnn_data = gnn_data.copy()
    np.random.shuffle(shuffled_gnn_data)
    shuffled_r2, _ = evaluate_model(model, mlp_data, shuffled_gnn_data, y_true)
    importance['GNN'] = baseline_r2 - shuffled_r2

    return importance
        
# ==================== Run the Analysis ==================== #

print("\n" + "="*80)
print(f"Analyzing GNN-MLP Autoencoder Model")
print("="*80)

batch_size = 4
gnn_input_dim = len(coords_train)
mlp_input_dim = mlp_train.shape[1]

model = build_gnn_mlp_autoencoder_model(mlp_input_dim, gnn_input_dim)
model.summary()

# ==================== 6. Create Data Generators ==================== #
train_generator = DataGenerator(
    mlp_data=mlp_train, gnn_data=gnn_train, y=y_train,
    batch_size=batch_size, shuffle=True
)

# ==================== 7. Train Model ==================== #
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=1,
    verbose=1,
    callbacks=[early_stopping],
    validation_data=train_generator
)

# ==================== 8. Evaluate & Perform Feature Importance ==================== #
# Predict on the training data using the generator
y_pred_train = model.predict(train_generator).flatten()
r2_train = r2_score(y_train[:len(y_pred_train)], y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train[:len(y_pred_train)], y_pred_train))

# Evaluate on the test data using the updated function
r2_test, rmse_test = evaluate_model(model, mlp_test, gnn_test, y_test)
y_pred_test = evaluate_model(model, mlp_test, gnn_test, y_test, return_preds=True)

print(f"\n GNN-MLP Autoencoder Model Performance:")
print(f"R² Train: {r2_train:.4f} | RMSE Train: {rmse_train:.4f}")
print(f"R² Test: {r2_test:.4f} | RMSE Test: {rmse_test:.4f}")

# Calculate and print feature importance
feature_importance = calculate_permutation_importance(model, mlp_test, gnn_test, y_test)
print("\n--- Feature Importance (Permutation) ---")
sorted_importance = sorted(feature_importance.items(), key=lambda item: item[1], reverse=True)
for feature, score in sorted_importance:
    print(f"{feature}: {score:.4f}")

# ==================== 9. Save all info to a folder ==================== #

output_folder = "gnn_mlp_ae"
os.makedirs(output_folder, exist_ok=True)
print(f"\nCreating folder: '{output_folder}' and saving results...")

# Save the model
model_path = os.path.join(output_folder, "gnn_mlp_ae_model.keras")
model.save(model_path)
print(f"Model saved to: {model_path}")

# Save the predictions and true labels
np.save(os.path.join(output_folder, "y_train.npy"), y_train)
np.save(os.path.join(output_folder, "y_test.npy"), y_test)
np.save(os.path.join(output_folder, "y_pred_train.npy"), y_pred_train)
np.save(os.path.join(output_folder, "y_pred_test.npy"), y_pred_test)
print(f"Predictions and true labels saved as .npy files.")

# Save the printed output to a text file
output_path = os.path.join(output_folder, "analysis_output.txt")
with open(output_path, "w") as f:
    f.write(printed_output)
print(f"Analysis results saved to: {output_path}")

# Save the feature importance dictionary as a .pkl file
importance_path = os.path.join(output_folder, "feature_importance.pkl")
with open(importance_path, 'wb') as f:
    pickle.dump(feature_importance, f)
print(f"Feature importance results saved to: {importance_path}")

print("\nAll information successfully saved.")

# Garbage collect to free up memory now that everything is saved
del model, history, train_generator
gc.collect()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ mlp_input           │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_input           │ (None, 110)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 128)       │      2,048 │ mlp_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │     14,208 │ gnn_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_encoder_output  │ (None, 64)        │      8,256 │ dense_4[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_encoder_output  │ (None, 64)        │      8,256 │ dense_5[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ latent_space        │ (None, 128)       │          0 │ mlp_encoder_outp… │
│ (Concatenate)       │                   │            │ gnn_encoder_outp… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │     16,512 │ latent_space[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 64)        │      8,256 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ final_output        │ (None, 1)         │         65 │ dense_7[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 57,601 (225.00 KB)

 Trainable params: 57,601 (225.00 KB)

 Non-trainable params: 0 (0.00 B)

6020

# AlphaEarth Integration Enabled

This notebook has been enhanced with AlphaEarth satellite embeddings.

## Integration Options:
- **Option A**: Replace indices with AlphaEarth (64 bands)
- **Option B**: Add AlphaEarth to features (RECOMMENDED)
- **Option C**: PCA-reduced AlphaEarth (20 components)
- **Option D**: MLP enhancement only

Expected improvement: +0.5% to +0.8% in R²

In [ ]:
# ==================== ALPHAEARTH CONFIGURATION ====================
import pandas as pd
import numpy as np
import os

# Select which AlphaEarth option to use
ALPHA_EARTH_OPTION = 'B'  # Options: A, B (recommended), C, D
USE_ALPHA_EARTH = True

# Paths to AlphaEarth data files (created by 00_AlphaEarth_Data_Preparation.ipynb)
option_file = f'Option_{ALPHA_EARTH_OPTION}_RainyAE.csv'  # or WinterAE

# Load AlphaEarth data
if os.path.exists(option_file):
    ae_data = pd.read_csv(option_file)
    print(f'Loaded AlphaEarth Option {ALPHA_EARTH_OPTION}')
    print(f'Shape: {ae_data.shape}')
else:
    print(f'WARNING: {option_file} not found')
    print('Please run 00_AlphaEarth_Data_Preparation.ipynb first')
    USE_ALPHA_EARTH = False


In [2]:
import pandas as pd
import numpy as np
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Dropout, Layer, MultiHeadAttention, LayerNormalization, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc # Import garbage collector
import sys
from io import StringIO
import pickle # Import the pickle library for saving objects
# Define the single buffer size to use
BUFFER_METERS = 500
# ==================== 1. Load Data ==================== #
# Note: The original script split data here. For k-fold, we load all data
# and perform the split later in the loop.
orig = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")
drop_cols = ['Stations','River','Lat','Long','geometry']
numeric_cols = orig.drop(columns=drop_cols).columns.drop('RI')
# ==================== 2. Collect ALL Rasters ==================== #
# We are not using rasters in this GNN-MLP model, but the paths are still
# defined for consistency with previous versions.
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")
print("Note: Raster data is not used in this GNN-MLP model.")
# ==================== 3. Create a Custom Data Generator ==================== #
class DataGenerator(Sequence):
    def __init__(self, mlp_data, gnn_data, y, batch_size=4, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))
        self.on_epoch_end()
    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        # Get batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        # Get batch data
        batch_mlp = self.mlp_data[batch_indices]
        batch_gnn = self.gnn_data[batch_indices, :]
        batch_y = self.y[batch_indices]
        
        return (batch_mlp, batch_gnn), batch_y
# ==================== 4. Define GNN-MLP Fusion Autoencoder Model ==================== #
def build_gnn_mlp_autoencoder_model(mlp_dim, gnn_dim):
    # Inputs for all branches
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    
    # --- Encoder Branch (MLP) ---
    mlp_encoded = Dense(128, activation="relu")(mlp_input)
    mlp_encoded = Dense(64, activation="relu", name="mlp_encoder_output")(mlp_encoded)
    # --- Encoder Branch (GNN) ---
    gnn_encoded = Dense(128, activation="relu")(gnn_input)
    gnn_encoded = Dense(64, activation="relu", name="gnn_encoder_output")(gnn_encoded)
    # --- Bottleneck/Latent Space ---
    # Concatenate the encoded representations
    latent_space = Concatenate(name="latent_space")([mlp_encoded, gnn_encoded])
    
    # --- Decoder Branch for Prediction ---
    # The decoder takes the latent space and performs the final prediction
    f = Dense(128, activation="relu")(latent_space)
    f = Dropout(0.4)(f)
    f = Dense(64, activation="relu")(f)
    output = Dense(1, activation="linear", name="final_output")(f)
    # Build and compile the model
    model = Model(inputs=[mlp_input, gnn_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model
# ==================== 5. Define New Evaluation Metrics ==================== #
def smape(y_true, y_pred):
    """
    Calculates the Symmetric Mean Absolute Percentage Error (SMAPE).
    """
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    # Add a small epsilon to avoid division by zero
    return np.mean(numerator / (denominator + 1e-8)) * 100
def evaluate_model(model, mlp_test, gnn_test_matrix, y_test, return_preds=False):
    """
    Evaluates the model on given data and returns R², RMSE, MAE, and SMAPE.
    """
    y_pred = model.predict((mlp_test, gnn_test_matrix)).flatten()
    
    if return_preds:
        return y_pred
    else:
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        smape_score = smape(y_test, y_pred)
        return r2, rmse, mae, smape_score
def calculate_permutation_importance(model, mlp_data, gnn_data, y_true):
    """
    Calculates permutation feature importance for the MLP and GNN branches.
    Uses R² as the scoring metric.
    """
    # Get baseline R² on the unshuffled data
    baseline_r2, _, _, _ = evaluate_model(model, mlp_data, gnn_data, y_true)
    importance = {}
    
    # Permute MLP input
    shuffled_mlp_data = mlp_data.copy()
    np.random.shuffle(shuffled_mlp_data)
    shuffled_r2, _, _, _ = evaluate_model(model, shuffled_mlp_data, gnn_data, y_true)
    importance['MLP'] = baseline_r2 - shuffled_r2
    # Permute GNN input
    shuffled_gnn_data = gnn_data.copy()
    np.random.shuffle(shuffled_gnn_data)
    shuffled_r2, _, _, _ = evaluate_model(model, mlp_data, shuffled_gnn_data, y_true)
    importance['GNN'] = baseline_r2 - shuffled_r2
    return importance
        
print("\n" + "="*80)
print(f"Analyzing GNN-MLP Autoencoder Model with 5-Fold Single Split")
print("="*80)
# Initialize lists to store metrics and predictions from each fold
r2_scores = []
rmse_scores = []
mae_scores = []
smape_scores = []
all_test_preds = np.array([])
all_test_y = np.array([])
importance_scores = {'MLP': [], 'GNN': []}
# Set up Train/Test Single Split
kf = KFold(n_splits=5, shuffle=True, random_state=42)
batch_size = 4
fold_count = 1
# Iterate through each fold
for train_index, test_index in kf.split(orig):
    print(f"\n--- Starting Fold {fold_count}/5 ---")
    
    # Create the training and test sets for this fold
    train_df = pd.concat([river_100, orig.iloc[train_index]], ignore_index=True)
    test_df = orig.iloc[test_index]
    
    # Scale data for this fold (important to prevent data leakage)
    scaler = StandardScaler()
    mlp_train = scaler.fit_transform(train_df[numeric_cols])
    mlp_test = scaler.transform(test_df[numeric_cols])
    y_train = train_df['RI'].values
    y_test = test_df['RI'].values
    
    # Calculate GNN inputs for this fold
    coords_train = train_df[['Long','Lat']].values
    coords_test = test_df[['Long','Lat']].values
    dist_mat_train = distance_matrix(coords_train, coords_train)
    gnn_train = np.exp(-dist_mat_train/10)
    dist_mat_test_train = distance_matrix(coords_test, coords_train)
    gnn_test = np.exp(-dist_mat_test_train/10)
    # Build and train the model for this fold
    gnn_input_dim = len(coords_train)
    mlp_input_dim = mlp_train.shape[1]
    model = build_gnn_mlp_autoencoder_model(mlp_input_dim, gnn_input_dim)
    
    # Create Data Generators for this fold
    train_generator = DataGenerator(
        mlp_data=mlp_train, gnn_data=gnn_train, y=y_train,
        batch_size=batch_size, shuffle=True
    )
    # Early stopping monitor on the test set for this fold
    early_stopping = EarlyStopping(
        monitor='loss', # Monitor training loss since we don't have a separate validation set
        patience=10,
        restore_best_weights=True
    )
    
    history = model.fit(
        train_generator,
        epochs=100,
        verbose=1,
        callbacks=[early_stopping]
    )
    
    # Evaluate model performance on this fold's test set
    r2, rmse, mae, smape_score = evaluate_model(model, mlp_test, gnn_test, y_test)
    r2_scores.append(r2)
    rmse_scores.append(rmse)
    mae_scores.append(mae)
    smape_scores.append(smape_score)
    print(f"Fold {fold_count} Results:")
    print(f"  R²: {r2:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  SMAPE: {smape_score:.4f}%")
    # Perform permutation importance for this fold
    importance = calculate_permutation_importance(model, mlp_test, gnn_test, y_test)
    importance_scores['MLP'].append(importance['MLP'])
    importance_scores['GNN'].append(importance['GNN'])
    
    print("\n  --- Feature Importance (Permutation) ---")
    sorted_importance = sorted(importance.items(), key=lambda item: item[1], reverse=True)
    for feature, score in sorted_importance:
        print(f"  {feature}: {score:.4f}")
    # Store test predictions and true values for later analysis
    y_pred_test_fold = evaluate_model(model, mlp_test, gnn_test, y_test, return_preds=True)
    all_test_preds = np.concatenate([all_test_preds, y_pred_test_fold])
    all_test_y = np.concatenate([all_test_y, y_test])
    # Clean up memory
    del model, train_generator, early_stopping
    gc.collect()
    fold_count += 1
    
# Calculate and print final cross-validation results
print("\n" + "="*80)
print("Final 5-Fold Single Split Results")
print("="*80)
print(f"Mean R²: {np.mean(r2_scores):.4f} (+/- {np.std(r2_scores):.4f})")
print(f"Mean RMSE: {np.mean(rmse_scores):.4f} (+/- {np.std(rmse_scores):.4f})")
print(f"Mean MAE: {np.mean(mae_scores):.4f} (+/- {np.std(mae_scores):.4f})")
print(f"Mean SMAPE: {np.mean(smape_scores):.4f}% (+/- {np.std(smape_scores):.4f}%)")
print("\nMean Permutation Feature Importance:")
print(f"  MLP: {np.mean(importance_scores['MLP']):.4f} (+/- {np.std(importance_scores['MLP']):.4f})")
print(f"  GNN: {np.mean(importance_scores['GNN']):.4f} (+/- {np.std(importance_scores['GNN']):.4f})")
# Save the mean scores and importance
results_dict = {
    'r2_scores': r2_scores,
    'rmse_scores': rmse_scores,
    'mae_scores': mae_scores,
    'smape_scores': smape_scores,
    'importance_scores': importance_scores,
}
results_dict


Note: Raster data is not used in this GNN-MLP model.

Analyzing GNN-MLP Autoencoder Model with 5-Fold Cross-Validation

--- Starting Fold 1/5 ---
Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 812us/step - loss: 35103.8008 
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 819us/step - loss: 30871.6641
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step - loss: 13082.8467
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step - loss: 5394.4341
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 886us/step - loss: 4508.8921
Epoch 6/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 883us/step - loss: 2615.6689
Epoch 7/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 885us/step - loss: 1022.8625
Epoch 8/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 883us/step - loss: 1056.6606
Epoch 9/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 898us/step - loss: 881.1158
Epoch 10/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 953us/step - loss: 876.2333
Epoch 11/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step - loss: 726.7940
Epoch 12/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 1111.718

{'r2_scores': [0.9571030556204042,
  0.9146838372840075,
  0.9632150298354079,
  0.9077061001059323,
  -0.01947207846939003],
 'rmse_scores': [9.916106948159937,
  34.46086946902181,
  10.913631634290637,
  24.629733009341436,
  14.878613125880559],
 'mae_scores': [9.12606781005859,
  28.252413864135743,
  9.11054829915364,
  21.019259134928376,
  12.663885904947918],
 'smape_scores': [6.971474805586787,
  15.99296405022412,
  5.695170893000018,
  9.92579720117381,
  10.521882639640252],
 'importance_scores': {'MLP': [2.3219447306938426,
   1.2152448275217171,
   -0.010451566970360093,
   2.3427269198509952,
   0.0],
  'GNN': [-0.0019177804228048823,
   0.00023859142754800722,
   0.0,
   -0.00029968478790243225,
   0.04903650551915506]}}

In [1]:
import pandas as pd
import numpy as np
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error # Added mean_absolute_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Dropout, Layer, MultiHeadAttention, LayerNormalization, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc # Import garbage collector
import sys
from io import StringIO
import pickle # Import the pickle library for saving objects

# Define the single buffer size to use
BUFFER_METERS = 500

# ==================== 1. Load Data ==================== #
orig = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")

drop_cols = ['Stations','River','Lat','Long','geometry']
numeric_cols = orig.drop(columns=drop_cols).columns.drop('RI')

# Train-test split
train_orig = orig.sample(10, random_state=42)
test_orig = orig.drop(train_orig.index)
train_combined = pd.concat([river_100, train_orig], ignore_index=True)

# ==================== 2. Collect ALL Rasters ==================== #
# We are not using rasters in this GNN-MLP model, but the paths are still
# defined for consistency with previous versions.
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")

print("Note: Raster data is not used in this GNN-MLP model.")

# ==================== 3. Create a Custom Data Generator ==================== #
class DataGenerator(Sequence):
    def __init__(self, mlp_data, gnn_data, y, batch_size=4, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        # Get batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Get batch data
        batch_mlp = self.mlp_data[batch_indices]
        batch_gnn = self.gnn_data[batch_indices, :]
        batch_y = self.y[batch_indices]
        
        return (batch_mlp, batch_gnn), batch_y

# ==================== 4. Prepare GNN & MLP Input (only once) ==================== #
coords_train = train_combined[['Long','Lat']].values
coords_test = test_orig[['Long','Lat']].values
dist_mat_train = distance_matrix(coords_train, coords_train)
gnn_train = np.exp(-dist_mat_train/10)
dist_mat_test_train = distance_matrix(coords_test, coords_train)
gnn_test = np.exp(-dist_mat_test_train/10)

scaler = StandardScaler()
mlp_train = scaler.fit_transform(train_combined[numeric_cols])
mlp_test = scaler.transform(test_orig[numeric_cols])
y_train = train_combined['RI'].values
y_test = test_orig['RI'].values

# ==================== 5. Define GNN-MLP Fusion Autoencoder Model ==================== #
def build_gnn_mlp_autoencoder_model(mlp_dim, gnn_dim):
    # Inputs for all branches
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    
    # --- Encoder Branch (MLP) ---
    mlp_encoded = Dense(128, activation="relu")(mlp_input)
    mlp_encoded = Dense(64, activation="relu", name="mlp_encoder_output")(mlp_encoded)

    # --- Encoder Branch (GNN) ---
    gnn_encoded = Dense(128, activation="relu")(gnn_input)
    gnn_encoded = Dense(64, activation="relu", name="gnn_encoder_output")(gnn_encoded)

    # --- Bottleneck/Latent Space ---
    # Concatenate the encoded representations
    latent_space = Concatenate(name="latent_space")([mlp_encoded, gnn_encoded])
    
    # --- Decoder Branch for Prediction ---
    # The decoder takes the latent space and performs the final prediction
    f = Dense(128, activation="relu")(latent_space)
    f = Dropout(0.4)(f)
    f = Dense(64, activation="relu")(f)
    output = Dense(1, activation="linear", name="final_output")(f)

    # Build and compile the model
    model = Model(inputs=[mlp_input, gnn_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    """
    Calculates the Symmetric Mean Absolute Percentage Error (SMAPE).
    """
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    # Handle the case where the denominator is zero
    mask = denominator == 0
    smape = np.where(mask, 0, numerator / denominator)
    return np.mean(smape) * 100

def evaluate_model(model, mlp_test, gnn_test_matrix, y_test, return_preds=False):
    """
    Evaluates the model on given data and returns R², RMSE, MAE, SMAPE, and predictions.
    """
    y_pred = model.predict((mlp_test, gnn_test_matrix)).flatten()
    
    if return_preds:
        return y_pred
    else:
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred) # Calculate MAE
        smape = symmetric_mean_absolute_percentage_error(y_test, y_pred) # Calculate SMAPE
        return r2, rmse, mae, smape

def calculate_permutation_importance(model, mlp_data, gnn_data, y_true):
    """
    Calculates permutation feature importance for the MLP and GNN branches.
    """
    print("\nStarting Permutation Feature Importance Analysis...")
    # Get baseline R² on the unshuffled data
    baseline_r2, _, _, _ = evaluate_model(model, mlp_data, gnn_data, y_true)
    print(f"Baseline R² on test set: {baseline_r2:.4f}")

    importance = {}
    
    # Permute MLP input
    shuffled_mlp_data = mlp_data.copy()
    np.random.shuffle(shuffled_mlp_data)
    shuffled_r2, _, _, _ = evaluate_model(model, shuffled_mlp_data, gnn_data, y_true)
    importance['MLP'] = baseline_r2 - shuffled_r2

    # Permute GNN input
    shuffled_gnn_data = gnn_data.copy()
    np.random.shuffle(shuffled_gnn_data)
    shuffled_r2, _, _, _ = evaluate_model(model, mlp_data, shuffled_gnn_data, y_true)
    importance['GNN'] = baseline_r2 - shuffled_r2

    return importance
        
# ==================== Run the Analysis ==================== #

print("\n" + "="*80)
print(f"Analyzing GNN-MLP Autoencoder Model")
print("="*80)

batch_size = 4
gnn_input_dim = len(coords_train)
mlp_input_dim = mlp_train.shape[1]

model = build_gnn_mlp_autoencoder_model(mlp_input_dim, gnn_input_dim)
model.summary()

# ==================== 6. Create Data Generators ==================== #
train_generator = DataGenerator(
    mlp_data=mlp_train, gnn_data=gnn_train, y=y_train,
    batch_size=batch_size, shuffle=True
)

# ==================== 7. Train Model ==================== #
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=100,
    verbose=1,
    callbacks=[early_stopping],
    validation_data=train_generator
)

# ==================== 8. Evaluate & Perform Feature Importance ==================== #
# Predict on the training data using the generator
y_pred_train = model.predict(train_generator).flatten()
r2_train = r2_score(y_train[:len(y_pred_train)], y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train[:len(y_pred_train)], y_pred_train))
mae_train = mean_absolute_error(y_train[:len(y_pred_train)], y_pred_train) # Calculate MAE
smape_train = symmetric_mean_absolute_percentage_error(y_train[:len(y_pred_train)], y_pred_train) # Calculate SMAPE

# Evaluate on the test data using the updated function
r2_test, rmse_test, mae_test, smape_test = evaluate_model(model, mlp_test, gnn_test, y_test)
y_pred_test = evaluate_model(model, mlp_test, gnn_test, y_test, return_preds=True)

print(f"\n GNN-MLP Autoencoder Model Performance:")
print(f"R² Train: {r2_train:.4f} | RMSE Train: {rmse_train:.4f} | MAE Train: {mae_train:.4f} | SMAPE Train: {smape_train:.4f}%")
print(f"R² Test: {r2_test:.4f} | RMSE Test: {rmse_test:.4f} | MAE Test: {mae_test:.4f} | SMAPE Test: {smape_test:.4f}%")

# Calculate and print feature importance
feature_importance = calculate_permutation_importance(model, mlp_test, gnn_test, y_test)
print("\n--- Feature Importance (Permutation) ---")
sorted_importance = sorted(feature_importance.items(), key=lambda item: item[1], reverse=True)
for feature, score in sorted_importance:
    print(f"{feature}: {score:.4f}")


Note: Raster data is not used in this GNN-MLP model.

Analyzing GNN-MLP Autoencoder Model


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ mlp_input           │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_input           │ (None, 110)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │      2,048 │ mlp_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     14,208 │ gnn_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_encoder_output  │ (None, 64)        │      8,256 │ dense[0][0]       │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_encoder_output  │ (None, 64)        │      8,256 │ dense_1[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ latent_space        │ (None, 128)       │          0 │ mlp_encoder_outp… │
│ (Concatenate)       │                   │            │ gnn_encoder_outp… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 128)       │     16,512 │ latent_space[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 64)        │      8,256 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ final_output        │ (None, 1)         │         65 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 57,601 (225.00 KB)

 Trainable params: 57,601 (225.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 31600.7109 - val_loss: 31203.8477
Epoch 2/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 28667.5371 - val_loss: 13123.4736
Epoch 3/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9901.9072 - val_loss: 3820.2981
Epoch 4/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3767.3232 - val_loss: 2783.6003
Epoch 5/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2883.7637 - val_loss: 1564.5033
Epoch 6/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2083.7891 - val_loss: 848.7328
Epoch 7/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 885.7454 - val_loss: 590.4202
Epoch 8/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1345.6345 - val_loss: 461.8258
Epoch 9/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 518.1317 - val_loss: 450.2736
Epoch 10/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 744.5951 - val_loss: 329.1303
Epoch 11/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 821.7020 - val_loss: 321.4229
Epo

#### This is also same fast as GNN MLP 

## Feature Importance

In [6]:
import pandas as pd
import numpy as np
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc
import sys
from io import StringIO
import pickle
import lime
from lime.lime_tabular import LimeTabularExplainer
from sklearn.ensemble import GradientBoostingRegressor


In [4]:

BUFFER_METERS = 500

# ==================== 1. Load Data ==================== #
# We assume the user has the correct data files in their project directory.
# For demonstration, we'll create dummy data to ensure the script is runnable.
orig = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")

drop_cols = ['Stations','River','Lat','Long','geometry']
numeric_cols = orig.drop(columns=drop_cols).columns.drop('RI')

# Train-test split
train_orig = orig.sample(10, random_state=42)
test_orig = orig.drop(train_orig.index)
train_combined = pd.concat([river_100, train_orig], ignore_index=True)

# ==================== 2. Collect ALL Rasters ==================== #
# We are not using rasters in this GNN-MLP model, but the paths are still
# defined for consistency with previous versions.
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")

print("Note: Raster data is not used in this GNN-MLP model.")

# ==================== 3. Create a Custom Data Generator ==================== #
class DataGenerator(Sequence):
    def __init__(self, mlp_data, gnn_data, y, batch_size=4, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        # Get batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Get batch data
        batch_mlp = self.mlp_data[batch_indices]
        batch_gnn = self.gnn_data[batch_indices, :]
        batch_y = self.y[batch_indices]
        
        return (batch_mlp, batch_gnn), batch_y

# ==================== 4. Prepare GNN & MLP Input (only once) ==================== #
coords_train = train_combined[['Long','Lat']].values
coords_test = test_orig[['Long','Lat']].values
dist_mat_train = distance_matrix(coords_train, coords_train)
gnn_train = np.exp(-dist_mat_train/10)
dist_mat_test_train = distance_matrix(coords_test, coords_train)
gnn_test = np.exp(-dist_mat_test_train/10)

scaler = StandardScaler()
mlp_train = scaler.fit_transform(train_combined[numeric_cols])
mlp_test = scaler.transform(test_orig[numeric_cols])
y_train = train_combined['RI'].values
y_test = test_orig['RI'].values

# ==================== 5. Define GNN-MLP Fusion Autoencoder Model ==================== #
def build_gnn_mlp_autoencoder_model(mlp_dim, gnn_dim):
    # Inputs for all branches
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    
    # --- Encoder Branch (MLP) ---
    mlp_encoded = Dense(128, activation="relu")(mlp_input)
    mlp_encoded = Dense(64, activation="relu", name="mlp_encoder_output")(mlp_encoded)

    # --- Encoder Branch (GNN) ---
    gnn_encoded = Dense(128, activation="relu")(gnn_input)
    gnn_encoded = Dense(64, activation="relu", name="gnn_encoder_output")(gnn_encoded)

    # --- Bottleneck/Latent Space ---
    # Concatenate the encoded representations
    latent_space = Concatenate(name="latent_space")([mlp_encoded, gnn_encoded])
    
    # --- Decoder Branch for Prediction ---
    # The decoder takes the latent space and performs the final prediction
    f = Dense(128, activation="relu")(latent_space)
    f = Dropout(0.4)(f)
    f = Dense(64, activation="relu")(f)
    output = Dense(1, activation="linear", name="final_output")(f)

    # Build and compile the model
    model = Model(inputs=[mlp_input, gnn_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    """
    Calculates the Symmetric Mean Absolute Percentage Error (SMAPE).
    """
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    # Handle the case where the denominator is zero
    mask = denominator == 0
    smape = np.where(mask, 0, numerator / denominator)
    return np.mean(smape) * 100

def evaluate_model(model, mlp_test, gnn_test_matrix, y_test, return_preds=False):
    """
    Evaluates the model on given data and returns R², RMSE, MAE, SMAPE, and predictions.
    """
    y_pred = model.predict((mlp_test, gnn_test_matrix)).flatten()
    
    if return_preds:
        return y_pred
    else:
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred) # Calculate MAE
        smape = symmetric_mean_absolute_percentage_error(y_test, y_pred) # Calculate SMAPE
        return r2, rmse, mae, smape

# ==================== Run the Analysis ==================== #

print("\n" + "="*80)
print(f"Analyzing GNN-MLP Autoencoder Model")
print("="*80)

batch_size = 4
gnn_input_dim = len(coords_train)
mlp_input_dim = mlp_train.shape[1]

model = build_gnn_mlp_autoencoder_model(mlp_input_dim, gnn_input_dim)

# ==================== 6. Create Data Generators ==================== #
train_generator = DataGenerator(
    mlp_data=mlp_train, gnn_data=gnn_train, y=y_train,
    batch_size=batch_size, shuffle=True
)

# ==================== 7. Train Model ==================== #
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=100,
    verbose=0,
    callbacks=[early_stopping],
    validation_data=train_generator
)


Note: Raster data is not used in this GNN-MLP model.

Analyzing GNN-MLP Autoencoder Model


In [7]:
# ==================== 8. Evaluate Model Performance ==================== #
# Predict on the training data using the generator
y_pred_train = model.predict(train_generator).flatten()
r2_train = r2_score(y_train[:len(y_pred_train)], y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train[:len(y_pred_train)], y_pred_train))
mae_train = mean_absolute_error(y_train[:len(y_pred_train)], y_pred_train)
smape_train = symmetric_mean_absolute_percentage_error(y_train[:len(y_pred_train)], y_pred_train)

# Evaluate on the test data using the updated function
r2_test, rmse_test, mae_test, smape_test = evaluate_model(model, mlp_test, gnn_test, y_test)
y_pred_test = evaluate_model(model, mlp_test, gnn_test, y_test, return_preds=True)

print(f"\n GNN-MLP Autoencoder Model Performance:")
print(f"R² Train: {r2_train:.4f} | RMSE Train: {rmse_train:.4f} | MAE Train: {mae_train:.4f} | SMAPE Train: {smape_train:.4f}%")
print(f"R² Test: {r2_test:.4f} | RMSE Test: {rmse_test:.4f} | MAE Test: {mae_test:.4f} | SMAPE Test: {smape_test:.4f}%")

# ==================== 9. Permutation Feature Importance ==================== #
# This is a model-agnostic method that measures importance by shuffling features.
# It gives a global view of which branches (MLP or GNN) are most important.
print("\n" + "="*80)
print("Performing Permutation Feature Importance")
print("="*80)

def calculate_permutation_importance(model, mlp_data, gnn_data, y_true):
    """
    Calculates permutation feature importance for the MLP and GNN branches.
    """
    print("\nStarting Permutation Feature Importance Analysis...")
    # Get baseline R² on the unshuffled data
    baseline_r2, _, _, _ = evaluate_model(model, mlp_data, gnn_data, y_true)
    print(f"Baseline R² on test set: {baseline_r2:.4f}")

    importance = {}
    
    # Permute MLP input
    shuffled_mlp_data = mlp_data.copy()
    np.random.shuffle(shuffled_mlp_data)
    shuffled_r2, _, _, _ = evaluate_model(model, shuffled_mlp_data, gnn_data, y_true)
    importance['MLP'] = baseline_r2 - shuffled_r2

    # Permute GNN input
    shuffled_gnn_data = gnn_data.copy()
    np.random.shuffle(shuffled_gnn_data)
    shuffled_r2, _, _, _ = evaluate_model(model, mlp_data, shuffled_gnn_data, y_true)
    importance['GNN'] = baseline_r2 - shuffled_r2

    return importance

feature_importance = calculate_permutation_importance(model, mlp_test, gnn_test, y_test)
print("\n--- Feature Importance (Permutation) ---")
sorted_importance = sorted(feature_importance.items(), key=lambda item: item[1], reverse=True)
for feature, score in sorted_importance:
    print(f"{feature}: {score:.4f}")

# ==================== 10. Intrinsic Feature Importance (Tree-based Model) ==================== #
# NOTE: Intrinsic importance is specific to tree-based models and cannot be
# applied to your TensorFlow model. This section demonstrates the concept by
# training a separate Gradient Boosting Regressor on just the MLP features.
print("\n" + "="*80)
print("Demonstrating Intrinsic Feature Importance with a Tree-based Model")
print("="*80)
print("Training a Gradient Boosting Regressor on the MLP features...")

# Create and train a tree-based model on the MLP features
gbr_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Fit the GBR model using the non-scaled MLP data from the training set
gbr_model.fit(train_combined[numeric_cols], train_combined['RI'].values)

# Get feature importances
intrinsic_importances = gbr_model.feature_importances_

# Print the results
print("\n--- Intrinsic Feature Importance from a Gradient Boosting Model ---")
for feature, importance in zip(numeric_cols, intrinsic_importances):
    print(f"{feature:<15}: {importance:.4f}")


# ==================== 11. LIME Feature Importance (Local Explanations) ==================== #
# LIME explains individual predictions by fitting a local, simple model.
# This is a powerful way to understand *why* a single data point was predicted a certain way.
print("\n" + "="*80)
print("Performing LIME-based Local Explanations on Test Set")
print("="*80)

# Create a wrapper function for the model's prediction
# LIME generates perturbed samples (a batch of new data)
# The GNN input is fixed for a given test point, so we tile the corresponding row
# of the gnn_test matrix to match the number of perturbed samples.
def predict_wrapper(data, test_point_index):
    # Separate the MLP and GNN data from the LIME input
    # Since LIME gives us a single numpy array, we need to split it
    # to match our model's dual-input structure.
    # The GNN data is not perturbed by LIME, so we need to grab the original
    # GNN data for the test point and replicate it for all perturbed samples.
    
    # LIME gives us data for the MLP features only, so we must manually add the GNN part
    scaled_data = scaler.transform(data)
    
    # The GNN part remains constant for this test point's explanation
    gnn_part = np.tile(gnn_test[test_point_index], (data.shape[0], 1))
    
    # Pass the inputs to the original model
    return model.predict((scaled_data, gnn_part)).flatten()

# Choose a random sample from the test set to explain
test_sample_index = np.random.randint(0, len(test_orig))
test_sample_data = test_orig[numeric_cols].iloc[test_sample_index].values
actual_ri = y_test[test_sample_index]

# Create a LIME explainer
# The explainer needs non-scaled training data and feature names
explainer = LimeTabularExplainer(
    training_data=train_combined[numeric_cols].values,
    feature_names=numeric_cols.to_list(),
    class_names=['RI'],
    mode='regression' # We are doing regression, not classification
)

# Generate and show the explanation for the chosen test sample
print(f"\nExplaining prediction for test sample at index: {test_sample_index}")
print(f"True RI value: {actual_ri:.4f}")

# The lambda function is a common way to pass a function with fixed arguments to a library
lime_explanation = explainer.explain_instance(
    data_row=test_sample_data,
    predict_fn=lambda x: predict_wrapper(x, test_sample_index),
    num_features=len(numeric_cols) # Explain all MLP features
)

# Print the explanation details
print("\nLIME Explanation:")
for feature, weight in lime_explanation.as_list():
    print(f"Feature: {feature:<15} | Weight: {weight:>10.4f}")


27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 836us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

 GNN-MLP Autoencoder Model Performance:
R² Train: -0.6483 | RMSE Train: 87.6643 | MAE Train: 66.6872 | SMAPE Train: 35.6327%
R² Test: 0.9746 | RMSE Test: 12.5989 | MAE Test: 11.4522 | SMAPE Test: 7.6491%

Performing Permutation Feature Importance

Starting Permutation Feature Importance Analysis...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Baseline R² on test set: 0.9746
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

--- Feature Importance (Permutation) ---
MLP: 1.4334
GNN: 0.0005

Demonstrating Intrinsic Feature Importance with a Tree-based Model
Training a Gradient Boosting Regressor on the MLP features...

--- Intrinsic Feature Importance from a Gradient Boosting Model ---
hydro_dist_brick: 0.0001
num_brick_field: 0.0000
hydro_dist_ind : 0.0012
num_industry   : 0.0000
CrR            : 0.0121
NiR            : 0.0988
CuR            : 0.0117
As

/Users/rakibhhridoy/anaconda3/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

LIME Explanation:
Feature: CdR <= 2.10     | Weight:   -11.0612
Feature: num_industry > 1.00 | Weight:    -5.1669
Feature: SandR > 46.99   | Weight:    -5.0592
Feature: AsR > 15.78     | Weight:    -4.3109
Feature: ClayR <= 23.23  | Weight:    -4.2326
Feature: SiltR <= 29.11  | Weight:     3.7679
Feature: 29.59 < NiR <= 40.56 | Weight:     3.4908
Feature: hydro_dist_ind <= 509.29 | Weight:    -2.7672
Feature: hydro_dist_brick <= 887.50 | Weight:     2.5258
Feature: num_brick_field > 1.00 | Weight:     1.7069
Feature: 31.75 < MR <= 33.17 | Weight:    -1.6101
Feature: 53.03 < CrR <= 67.08 | Weight:     1.3195
Feature: 32253.00 < FeR <= 41226.06 | Weight:     0.8557
Feature: 69.42 < CuR <= 82.96 | Weight:    -0.5971
Feature: 51.99 < PbR <= 99.19 | Weight:    -0.5325


In [3]:
import pandas as pd
import numpy as np
import glob
import os
import rasterio
from rasterio.windows import Window
from scipy.spatial import distance_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import gc
import sys
from io import StringIO
import pickle

# Define the single buffer size to use
BUFFER_METERS = 500

# ==================== 1. Load Data ==================== #
orig = pd.read_csv("../../data/RainySeason.csv")
river_100 = pd.read_csv("../data/Samples_100.csv")

drop_cols = ['Stations','River','Lat','Long','geometry']
numeric_cols = orig.drop(columns=drop_cols).columns.drop('RI')

# Train-test split
train_orig = orig.sample(10, random_state=42)
test_orig = orig.drop(train_orig.index)
train_combined = pd.concat([river_100, train_orig], ignore_index=True)

# ==================== 2. Collect ALL Rasters ==================== #
# We are not using rasters in this GNN-MLP model, but the paths are still
# defined for consistency with previous versions.
raster_paths = []
raster_paths += glob.glob("../CalIndices/*.tif")
raster_paths += glob.glob("../LULCMerged/*.tif")
raster_paths += glob.glob("../IDW/*.tif")

print("Note: Raster data is not used in this GNN-MLP model.")

# ==================== 3. Create a Custom Data Generator ==================== #
class DataGenerator(Sequence):
    def __init__(self, mlp_data, gnn_data, y, batch_size=4, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.mlp_data = mlp_data
        self.gnn_data = gnn_data
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.y))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def __getitem__(self, index):
        # Get batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Get batch data
        batch_mlp = self.mlp_data[batch_indices]
        batch_gnn = self.gnn_data[batch_indices, :]
        batch_y = self.y[batch_indices]
        
        return (batch_mlp, batch_gnn), batch_y

# ==================== 4. Prepare GNN & MLP Input (only once) ==================== #
coords_train = train_combined[['Long','Lat']].values
coords_test = test_orig[['Long','Lat']].values
dist_mat_train = distance_matrix(coords_train, coords_train)
gnn_train = np.exp(-dist_mat_train/10)
dist_mat_test_train = distance_matrix(coords_test, coords_train)
gnn_test = np.exp(-dist_mat_test_train/10)

scaler = StandardScaler()
mlp_train = scaler.fit_transform(train_combined[numeric_cols])
mlp_test = scaler.transform(test_orig[numeric_cols])
y_train = train_combined['RI'].values
y_test = test_orig['RI'].values

# ==================== 5. Define GNN-MLP Fusion Autoencoder Model ==================== #
def build_gnn_mlp_autoencoder_model(mlp_dim, gnn_dim):
    # Inputs for all branches
    mlp_input = Input(shape=(mlp_dim,), name="mlp_input")
    gnn_input = Input(shape=(gnn_dim,), name="gnn_input")
    
    # --- Encoder Branch (MLP) ---
    mlp_encoded = Dense(128, activation="relu")(mlp_input)
    mlp_encoded = Dense(64, activation="relu", name="mlp_encoder_output")(mlp_encoded)

    # --- Encoder Branch (GNN) ---
    gnn_encoded = Dense(128, activation="relu")(gnn_input)
    gnn_encoded = Dense(64, activation="relu", name="gnn_encoder_output")(gnn_encoded)

    # --- Bottleneck/Latent Space ---
    # Concatenate the encoded representations
    latent_space = Concatenate(name="latent_space")([mlp_encoded, gnn_encoded])
    
    # --- Decoder Branch for Prediction ---
    # The decoder takes the latent space and performs the final prediction
    f = Dense(128, activation="relu")(latent_space)
    f = Dropout(0.4)(f)
    f = Dense(64, activation="relu")(f)
    output = Dense(1, activation="linear", name="final_output")(f)

    # Build and compile the model
    model = Model(inputs=[mlp_input, gnn_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss="mse")
    return model

def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    """
    Calculates the Symmetric Mean Absolute Percentage Error (SMAPE).
    """
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    # Handle the case where the denominator is zero
    mask = denominator == 0
    smape = np.where(mask, 0, numerator / denominator)
    return np.mean(smape) * 100

def evaluate_model(model, mlp_test, gnn_test_matrix, y_test, return_preds=False):
    """
    Evaluates the model on given data and returns R², RMSE, MAE, SMAPE, and predictions.
    """
    y_pred = model.predict((mlp_test, gnn_test_matrix)).flatten()
    
    if return_preds:
        return y_pred
    else:
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        smape = symmetric_mean_absolute_percentage_error(y_test, y_pred)
        return r2, rmse, mae, smape

def calculate_permutation_importance(model, mlp_data, gnn_data, y_true, mlp_feature_names):
    """
    Calculates permutation feature importance for the MLP and GNN branches,
    as well as individual MLP features.
    """
    print("\nStarting Permutation Feature Importance Analysis...")
    # Get baseline R² on the unshuffled data
    baseline_r2, _, _, _ = evaluate_model(model, mlp_data, gnn_data, y_true)
    print(f"Baseline R² on test set: {baseline_r2:.4f}")

    importance = {}
    
    # Permute GNN input (full branch)
    shuffled_gnn_data = gnn_data.copy()
    np.random.shuffle(shuffled_gnn_data)
    shuffled_r2, _, _, _ = evaluate_model(model, mlp_data, shuffled_gnn_data, y_true)
    importance['GNN_Branch_Overall'] = baseline_r2 - shuffled_r2

    # Permute MLP input (full branch)
    shuffled_mlp_data = mlp_data.copy()
    np.random.shuffle(shuffled_mlp_data)
    shuffled_r2, _, _, _ = evaluate_model(model, shuffled_mlp_data, gnn_data, y_true)
    importance['MLP_Branch_Overall'] = baseline_r2 - shuffled_r2
    
    # Permute each individual MLP feature
    for i, feature_name in enumerate(mlp_feature_names):
        # Create a copy to shuffle
        shuffled_mlp_data_single = mlp_data.copy()
        # Shuffle the current column
        np.random.shuffle(shuffled_mlp_data_single[:, i])
        
        # Evaluate with the shuffled column
        shuffled_r2, _, _, _ = evaluate_model(model, shuffled_mlp_data_single, gnn_data, y_true)
        
        # Calculate importance
        importance[feature_name] = baseline_r2 - shuffled_r2

    return importance
        
# ==================== Run the Analysis ==================== #

print("\n" + "="*80)
print(f"Analyzing GNN-MLP Autoencoder Model")
print("="*80)

batch_size = 4
gnn_input_dim = len(coords_train)
mlp_input_dim = mlp_train.shape[1]

model = build_gnn_mlp_autoencoder_model(mlp_input_dim, gnn_input_dim)

# ==================== 6. Create Data Generators ==================== #
train_generator = DataGenerator(
    mlp_data=mlp_train, gnn_data=gnn_train, y=y_train,
    batch_size=batch_size, shuffle=True
)

# ==================== 7. Train Model ==================== #
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    epochs=100,
    verbose=0,
    callbacks=[early_stopping],
    validation_data=train_generator
)

# ==================== 8. Evaluate & Perform Feature Importance ==================== #
# Predict on the training data using the generator
y_pred_train = model.predict(train_generator).flatten()
r2_train = r2_score(y_train[:len(y_pred_train)], y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train[:len(y_pred_train)], y_pred_train))
mae_train = mean_absolute_error(y_train[:len(y_pred_train)], y_pred_train)
smape_train = symmetric_mean_absolute_percentage_error(y_train[:len(y_pred_train)], y_pred_train)

# Evaluate on the test data using the updated function
r2_test, rmse_test, mae_test, smape_test = evaluate_model(model, mlp_test, gnn_test, y_test)
y_pred_test = evaluate_model(model, mlp_test, gnn_test, y_test, return_preds=True)

print(f"\n GNN-MLP Autoencoder Model Performance:")
print(f"R² Train: {r2_train:.4f} | RMSE Train: {rmse_train:.4f} | MAE Train: {mae_train:.4f} | SMAPE Train: {smape_train:.4f}%")
print(f"R² Test: {r2_test:.4f} | RMSE Test: {rmse_test:.4f} | MAE Test: {mae_test:.4f} | SMAPE Test: {smape_test:.4f}%")

# Calculate and print feature importance
feature_importance = calculate_permutation_importance(model, mlp_test, gnn_test, y_test, numeric_cols)
print("\n--- Feature Importance (Permutation) ---")
sorted_importance = sorted(feature_importance.items(), key=lambda item: item[1], reverse=True)
for feature, score in sorted_importance:
    print(f"{feature}: {score:.6f}")


Note: Raster data is not used in this GNN-MLP model.

Analyzing GNN-MLP Autoencoder Model
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 544us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

 GNN-MLP Autoencoder Model Performance:
R² Train: -1.1335 | RMSE Train: 99.7340 | MAE Train: 75.3063 | SMAPE Train: 39.8088%
R² Test: 0.9547 | RMSE Test: 16.8303 | MAE Test: 14.5286 | SMAPE Test: 9.9112%

Starting Permutation Feature Importance Analysis...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Baseline R² on test set: 0.9547
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━